In [3]:
import pandas as pd

In [5]:
!wget https://raw.githubusercontent.com/akshaybhalotia/yc_company_scraper/main/data/yc_essential_data.csv

--2023-11-30 19:51:18--  https://raw.githubusercontent.com/akshaybhalotia/yc_company_scraper/main/data/yc_essential_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4173693 (4.0M) [text/plain]
Saving to: ‘yc_essential_data.csv’

yc_essential_data.c 100%[===================>]   3.98M  --.-KB/s    in 0.08s   

2023-11-30 19:51:19 (50.3 MB/s) - ‘yc_essential_data.csv’ saved [4173693/4173693]



In [11]:
d = pd.read_csv('yc_essential_data.csv')

In [12]:
d = d[['long_description']]
d = d.dropna()

In [26]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv


df = d

#Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
df = df[df['long_description'].apply(lambda x: 200 < len(x) < 2000)]

#Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 200)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['true_end'] = test_set['long_description'].str.split().str[-10:].apply(' '.join)
test_set['long_description'] = test_set['long_description'].str.split().str[:-10].apply(' '.join)

In [27]:
class StartUpDescription(Dataset):
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['long_description']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
        if truncate:
            self.lyrics = self.lyrics[:2000]
        self.lyrics_count = len(self.lyrics)

    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

dataset = StartUpDescription(df['long_description'], truncate=True, gpt2_type="gpt2")

In [15]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [63]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=10, lr=1e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [64]:
device = "cuda"
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [65]:
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


2000it [02:36, 12.78it/s]


Training epoch 1
tensor(1.6841, device='cuda:0', grad_fn=<NllLossBackward0>)


2000it [02:34, 12.91it/s]


...

2000it [02:35, 12.88it/s]


Training epoch 8
tensor(1.1351, device='cuda:0', grad_fn=<NllLossBackward0>)


2000it [02:34, 12.96it/s]


Training epoch 9
tensor(1.2217, device='cuda:0', grad_fn=<NllLossBackward0>)


2000it [02:34, 12.93it/s]


In [107]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=2000, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list


In [28]:
test_set.dropna()

,index,long_description,true_end
0,1640,"Founded in 2005, TextPayMe is an SMS payment s...",that can be sent or received per month is $500.
1,2041,Nirva offers a comprehensive solution for wome...,Nirva's clients have reported improvement in s...
2,3328,RentHop (www.renthop.com) helps renters find g...,"also find us on Facebook, Twitter (@RentHop), ..."
3,2086,Tupelo is building the future of how tradition...,to make selling and buying a business quick an...
4,680,BloomAPI simplifies how medical records are tr...,needed for the care of patients. We make this ...
...,...,...,...
195,1103,Nobody should have to employ expensive imaging...,"Kickstarter, AirBnb, Coursera, Bonobos, Spotif..."
196,1516,Relativity is building humanity’s multiplaneta...,to upgrade humanity’s industrial base on Earth...
197,1184,"SoundFocus is a San Francisco, CA-based startu...",and August Capital Partners of Singapore. Hear...
198,1851,REGENT is building a new mode of transportatio...,"strategic investment from Japan Airlines, Hawa..."


In [69]:
import pickle

In [70]:
pickle.dump(model, open('model.pkl', 'wb'))

In [39]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['long_description'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

#Run the functions to generate the lyrics
generated_lyrics = text_generation(test_set[:50])

100%|██████████| 1/1 [00:03<00:00,  3.48s/it]


In [41]:
test_set = test_set[:50]

In [42]:
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['long_description'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['new_description'] = my_generations


#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['new_description'][i].split('.')[-1]
  final.append(test_set['new_description'][i].replace(to_remove,''))

test_set['new_description'] = final

<ipython-input-42-60d7d69e335b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['new_description'] = my_generations
<ipython-input-42-60d7d69e335b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['new_description'] = final


In [45]:
test_set['new_combined'] = test_set['long_description'] + test_set['new_description']

<ipython-input-45-d7e967c63d00>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['new_combined'] = test_set['long_description'] + test_set['new_description']


In [49]:
test_set['long_description'][0]

'Founded in 2005, TextPayMe is an SMS payment service that lets you send money to other people using cell phone text messaging. TextPayMe lets you send money to anyone else in the US with an SMS-capable cell phone, and while phone carriers may charge for the actual text message sent/received, sending and receiving money is free while TextPayMe remains in Beta. In July 2006, the TextPayMe team started working for Amazon and, as of 2011, the site redirects to an Amazon\'s mobile payment website. There was no press release related to this event but it\'s assumed the company was acquired. Sending and receiving payments through TextPayMe is easy: simply write a text message to SMS@TextPayMe.com with "pay amount phonenumber" in the body (i.e. to send $5 to 123-456-7890, text "pay 5 1234567890"). The service is secure as well, as each payment needs to be confirmed by a user-created PIN. To fund your transactions, TextPayMe lets you add money directly from your bank account and now also lets y

In [48]:
test_set['new_combined'][0]

'Founded in 2005, TextPayMe is an SMS payment service that lets you send money to other people using cell phone text messaging. TextPayMe lets you send money to anyone else in the US with an SMS-capable cell phone, and while phone carriers may charge for the actual text message sent/received, sending and receiving money is free while TextPayMe remains in Beta. In July 2006, the TextPayMe team started working for Amazon and, as of 2011, the site redirects to an Amazon\'s mobile payment website. There was no press release related to this event but it\'s assumed the company was acquired. Sending and receiving payments through TextPayMe is easy: simply write a text message to SMS@TextPayMe.com with "pay amount phonenumber" in the body (i.e. to send $5 to 123-456-7890, text "pay 5 1234567890"). The service is secure as well, as each payment needs to be confirmed by a user-created PIN. To fund your transactions, TextPayMe lets you add money directly from your bank account and now also lets y

In [90]:
start_one = "New startup, LeeMonade, introduce you our new refreshing drink for you"

In [91]:
start_two = "What do you know about Tyergrth? We provide you with the newest way to do the transactions"

In [92]:
new_test_set = pd.DataFrame([start_one, start_two], columns=['long_description'])

In [93]:
generated_lyrics = text_generation(new_test_set)

100%|██████████| 1/1 [00:04<00:00,  4.08s/it]


In [95]:
my_generations=[]

for i in range(len(generated_lyrics)):
  a = new_test_set['long_description'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

new_test_set['new_description'] = my_generations


#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(new_test_set)):
  to_remove = new_test_set['new_description'][i].split('.')[-1]
  final.append(new_test_set['new_description'][i].replace(to_remove,''))

new_test_set['new_description'] = new_test_set['long_description'] + new_test_set['new_description']

In [96]:
new_test_set['new_description'][0]

'New startup, LeeMonade, introduce you our new refreshing drink for you to enjoy with all your friends, coworkers and family. No more fuss, no more wasted money. We give you a hard time with even the simplest of menus and an easy way to get through it.<|endoftext|>'

In [77]:
new_test_set['new_description'][1]

"What do you know about Tyergrth? We provide you with the newest way to do the transactions you need to connect your account to your real bank account. In turn, you'll never have to worry about having your account blocked by a firewall or other means.\n\nHow to use Tiers\n\n1. Sign up for Tiers\n\n1. In your Account page, scroll down to Buy and sell and click signup. Choose a Store or come back to the Marketplace.\n\n2. Tap on the Unique Products list. In the top-right corner, click the product<|endoftext|>"

In [104]:
startup_three = "Woptify Spotted provides with all the data concerning your music taste. You can get new statistics, insignts and suggestions that help you find your new favourite tracks. Moreover,"

In [105]:
startup_four = "Latte Art AI was designed for you to create the most beautiful drawings on your frothy latte with the help of AI. The possibilities are insane: you can"

In [106]:
new_test_set = pd.DataFrame([startup_three, startup_four], columns=['long_description'])

In [108]:
generated_lyrics = text_generation(new_test_set)

100%|██████████| 1/1 [02:32<00:00, 152.83s/it]


In [109]:
my_generations=[]

for i in range(len(generated_lyrics)):
  a = new_test_set['long_description'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

new_test_set['new_description'] = my_generations


#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(new_test_set)):
  to_remove = new_test_set['new_description'][i].split('.')[-1]
  final.append(new_test_set['new_description'][i].replace(to_remove,''))

new_test_set['new_description'] = new_test_set['long_description'] + new_test_set['new_description']

In [110]:
new_test_set['new_description'][0]

'Woptify Spotted provides with all the data concerning your music taste. You can get new statistics, insignts and suggestions that help you find your new favourite tracks. Moreover, Spotted is free to use so you can use it all with other people.<|endoftext|>'

In [111]:
new_test_set['new_description'][1]

'Latte Art AI was designed for you to create the most beautiful drawings on your frothy latte with the help of AI. The possibilities are insane: you can add colors, enhance textures, add layers, repeat, and edit the drawings. There are many games, and games you can play and play with AI – but there is no such thing as "time travel." At Microsoft, we believe you are going to enjoy programming, like this "Magic Hat" on your computer.\n\nWe created this magic hat so you can use it to enjoy and learn on your computer at home and at work, no matter where you live. We understand you want it to become a part of your daily life.\n\nMade from the material that came with your computer, we make every effort to make your ideas and experiences as comfortable as possible. We use our own proprietary AI technology to generate a simplified interface. This enables you to open your computer to a video editor and visual editor on your computer screen.\n\nIn addition to making that experience more comforta

In [112]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [113]:
pick_insert = open('drive/My Drive/model.pkl','wb')
pickle.dump(model, pick_insert)
pick_insert.close()